## JsonOutputParser

JsonOutputParser는 사용자가 원하는 JSON 스키마를 지정할 수 있게 해주는 도구. 이 도구는 Large Language Model(LLM)이 데이터를 조회하고 결과를 도출할 때, 지정된 스키마에 맞게 JSON 형식으로 데이터를 반환할 수 있도록 설계

LLM이 데이터를 정확하고 효율적으로 처리하여 사용자가 원하는 형태의 JSON을 생성하기 위해서, 모델의 용량이 충분히 커야 한다. 예를 들어 llama-70B 모델은 llama-8B 모델보다 더 큰 용량을 가지고 있어 보다 복잡한 데이터를 처리하는 데 유리.


In [1]:
from dotenv import load_dotenv
from langchain_teddynote import logging

# .env 파일 로드
load_dotenv()

# langSmith에 로깅 할 프로젝트 명을 입력
logging.langsmith("03-OutputParser")

LangSmith 추적을 시작합니다.
[프로젝트명]
03-OutputParser


In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

In [4]:
# OpenAI 객체를 생성
model = ChatOpenAI(temperature=0, model_name="gpt-4o")

원하는 출력 구조를 정의한다

In [5]:
# 원하는 데이터 구조를 정의한다
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtag: str = Field(description="해시태그 형식의 키워드(2개 이상)")

`JsonOutputParser`를 사용하여 파서를 설정하고, 프롬프트 템플릿에 지시사항을 주입.

In [6]:
# 질의 작성
question = "지구 온난화의 심각성에 대해 알려주세요."

# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입
parser = JsonOutputParser(pydantic_object=Topic)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"description": {"description": "주제에 대한 간결한 설명", "title": "Description", "type": "string"}, "hashtag": {"description": "해시태그 형식의 키워드(2개 이상)", "title": "Hashtag", "type": "string"}}, "required": ["description", "hashtag"]}
```


In [7]:
# 프롬프트 템플릿을 설정
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변해주세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}")
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

# 체인을 구성
chain = prompt | model | parser

# 체인을 호출하여 쿼리 실행
answer = chain.invoke({"question": question})

In [9]:
print(answer)

{'description': '지구 온난화는 지구의 평균 기온이 상승하는 현상으로, 이는 주로 인간 활동에 의해 발생하는 온실가스 배출 증가로 인해 발생합니다. 이로 인해 극지방의 빙하가 녹고 해수면이 상승하며, 기후 패턴이 변화하여 극단적인 기상 현상이 빈번해지고 있습니다.', 'hashtag': '#지구온난화 #기후변화 #환경보호'}


In [ ]:
print(answer['hashtag'])

#지구온난화 #기후변화 #환경보호


In [10]:
type(answer)

dict